# 1.
Создайте расширенные профили посетителей магазина «Читать не вредно».
Для этого измените функцию get_profiles() из теории так, чтобы кроме источника перехода на сайт она возвращала устройство (device) и регион (region) первого посещения.
Передайте обновлённой функции данные из файла sessions.csv.
Затем определите, сколько пользователей заходили на сайт с каждого устройства.
Результат отсортируйте по убыванию количества посетителей и выведите на экран.
Структуру sessions.csv можно посмотреть во вкладке.

Подсказка
В теле функции get_profiles() передайте методу agg() два дополнительных аргумента: примените функцию first() к значениям столбцов device и region.
Чтобы узнать количество пользователей, зашедших на сайт с каждого устройства, вызовите метод agg() и примените функцию nunique() к значениям столбца user_id.
Отсортируйте результат по убыванию, вызвав метод sort_values() с такими параметрами: by = 'user_id', ascending = False.

In [2]:
import pandas as pd

sessions = pd.read_csv(filepath_or_buffer='datasets/sessions.csv')
sessions['session_start'] = pd.to_datetime(sessions['session_start'])


def get_profiles(sessions):

    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
            .groupby('user_id')
            .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device':'first',
                'region':'first'
            }
        )
            .rename(columns={'session_start': 'first_ts'})
            .reset_index()
    )

    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    return profiles


profiles = get_profiles(sessions)

print(
    profiles.groupby('device')
    .agg({'user_id':'nunique'})
    .sort_values(by='user_id', ascending=False)# допишите код
)

         user_id
device          
iPhone     62944
Mac        35354
Android    28390
PC         14731


# 2.
В дополнение к журналу пользовательских сессий интернет-магазин «Читать не вредно» передал вам файл с данными о всех покупках пользователей — book_orders.csv.
Вот его структура:

    user_id — уникальный идентификатор пользователя,
    event_dt — дата покупки,
    revenue — сумма покупки.

Обогатите пользовательские профили информацией о покупках: добавьте в профили признак payer, который принимает значение True, если идентификатор пользователя есть среди покупателей, и значение False в ином случае.
После этого посчитайте, какая доля новых посетителей стала покупателями в каждой стране.
Результат отсортируйте по убыванию доли покупателей и выведите на экран.

Подсказка
Чтобы добавить в столбец payer результаты проверки наличия user_id из profiles в данных orders, примените к столбцу user_id из датафрейма profiles метод isin(), передав ему уникальные значения столбца user_id из датафрейма orders.
Уникальные значения найдите методом unique().
Долю покупателей в каждой стране посчитайте как среднее по колонке payer: после группировки по столбцу region вызовите метод agg() и примените функцию mean() к значениям столбца payer.
Затем отсортируйте значения по убыванию значений payer методом sort_values().

In [ ]:
import pandas as pd

sessions = pd.read_csv('sessions.csv')
sessions['session_start'] = pd.to_datetime(sessions['session_start'])

orders = pd.read_csv('book_orders.csv')
orders['event_dt'] = pd.to_datetime(orders['event_dt'])

# добавили второй аргумент
def get_profiles(sessions, orders):

    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
            .groupby('user_id')
            .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
            .rename(columns={'session_start': 'first_ts'})
            .reset_index()
    )

    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # проверьте, есть ли ID пользователей из profiles в orders
    profiles['payer'] =  profiles['user_id'].isin(orders['user_id'])# ваш код здесь

    return profiles

# строим профили по двум наборам данных
profiles = get_profiles(sessions, orders)

print(
    profiles.groupby('region')
        .agg({'payer':'mean'})
        .sort_values(by='payer', ascending=False)
)